<a href="https://colab.research.google.com/github/suha1419/FinalProjectCSPB4380/blob/main/CSPB4380FinalProject_suha1419.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#pip install transformers datasets scikit-learn torch evaluate accelerate
#!pip install -U "transformers>=4.30.0" "datasets" "evaluate" "accelerate" "scikit-learn" "torch" "scikit-learn"
!pip install "transformers>=4.30.0"
!pip install datasets
!pip install torch
!pip install scikit-learn
!pip install evaluate
!pip install accelerate

In [ ]:
# Step 2: Imports
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_recall_fscore_support, confusion_matrix
import evaluate
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import torch
from torch.utils.data import DataLoader
from torch.nn.functional import softmax
from tqdm import tqdm
import torch.nn as nn


In [ ]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

In [ ]:
# Load Dataset
# For financial phrase bank, labels are:
# 0: Negative
# 1: Neutral
# 2: Positive
# We will maintain this structure

dataset = load_dataset("financial_phrasebank", "sentences_50agree")
print(dataset["train"][0])
print(dataset["train"][1])
print(dataset["train"][2])
print(dataset["train"].features)

In [ ]:
# Load Models
# Note: we instantiate the bert-base with 3 labels
models = {
    "bert-base": {
        "model": AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3),
        "tokenizer": AutoTokenizer.from_pretrained("bert-base-uncased")
    },
    "finbert": {
        "model": AutoModelForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone"),
        "tokenizer": AutoTokenizer.from_pretrained("yiyanghkust/finbert-tone")
    }
}

In [ ]:
# Check model configuration
# Note: We need to remap the finbert output to the model dataset for consistency
# bert has not been trained
print(models["bert-base"]["model"].config.id2label)
print(models["finbert"]["model"].config.id2label)

In [ ]:
# Split the dataset
# Test size of 20%, shuffle data, stratify to maintain proportions

train_texts, val_texts, train_labels, val_labels = train_test_split(
    dataset['train']['sentence'],
    dataset['train']['label'],
    test_size=0.2,
    shuffle=True,
    stratify=dataset['train']['label'],
    random_state=42)

print(f'Number of training examples: {len(train_texts)}')
print(f'Number of validation examples: {len(val_texts)}')

#train_labels_finbert = remap_labels_for_finbert(train_labels)
#val_labels_finbert = remap_labels_for_finbert(val_labels)

In [ ]:
# Tokenize Data
# Use each model's tokenizer
# deprecated - using in the SentimentDataset wrapper

def tokenize_data(tokenizer, dataset):
  return tokenizer(dataset, padding=True, truncation=True, max_length=128)

#train_encodings_bert = tokenize_data(models["bert-base"]["tokenizer"], train_texts)
#val_encodings_bert = tokenize_data(models["bert-base"]["tokenizer"], val_texts)
#train_encodings_finbert = tokenize_data(models["finbert"]["tokenizer"], train_texts)
#val_encodings_finbert = tokenize_data(models["finbert"]["tokenizer"], val_texts)


# Build Remap for Finbert
def remap_finbert_to_phrasebank(labels):
  # Remap finbert labels to match financial phrasebank dataset
  # Finbert -> Phrasebank
  # 0 ->  1 (Neutral)
  # 1 -> 2 (Positive)
  # 2 -> 0 (Negative)
  label_map = {0:1, 1:2, 2:0}
  return [label_map[label] for label in labels]

In [ ]:
test = [1,2,0] #pos, neg, neu
remap_finbert_to_phrasebank(test)

# 2,0,1 in phrasebank is: pos, neg, neutral

In [ ]:
# Create wrapper for DataLoader
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=max_len)
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)





In [ ]:
# From Hugging Face Tutorial
#import torch
#from transformers import AutoTokenizer, AutoModelForSequenceClassification

#checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
#tokenizer = AutoTokenizer.from_pretrained(checkpoint)
#model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
#sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

#tokens = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")
#output = model(**tokens)

In [ ]:
def evaluate_model(model, dataloader, model_name=None, device="cuda" if torch.cuda.is_available() else "cpu"):
    model.eval()
    model.to(device)

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc=f"Evaluating {model_name or 'model'}"):

            # Extract Batch Features
            input_ids      = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels         = batch['labels'].to(device)

            # Run through model for predictions
            outputs     = model(input_ids=input_ids, attention_mask=attention_mask)
            logits      = outputs.logits
            predictions = torch.argmax(logits, dim=1)

            # Remap predictions from FinBERT head to PhraseBank if needed
            if model_name == "finbert":
                remapped = remap_finbert_to_phrasebank(predictions.cpu().tolist())
                predictions = torch.tensor(remapped, device=device)

            # Move results to CPU for scikit
            all_preds.extend(predictions.cpu().tolist())
            all_labels.extend(labels.cpu().tolist())


    # Compute metrics
    label_names = ["Negative", "Neutral", "Positive"]
    accuracy = sum([p == l for p, l in zip(all_preds, all_labels)]) / len(all_labels)

    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, labels=[0, 1, 2], average="macro", zero_division=0)

    conf_matrix = confusion_matrix(all_labels, all_preds, labels=[0, 1, 2])

    class_report = classification_report(all_labels, all_preds, target_names=label_names, zero_division=0)

    # Print Results
    print("Classification Report:")
    print(class_report)

    print("Confusion Matrix:")
    print(conf_matrix)

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "confusion_matrix": conf_matrix
    }

In [ ]:
resFin = evaluate_model(models["finbert"]["model"], DataLoader(SentimentDataset(train_texts, train_labels, models["finbert"]["tokenizer"]), batch_size=16), model_name="finbert")

In [ ]:
resBert = evaluate_model(models["bert-base"]["model"], DataLoader(SentimentDataset(train_texts, train_labels, models["bert-base"]["tokenizer"]), batch_size=16))

In [ ]:
def train_bert(model, train_dataloader, val_dataloader, epochs=3, lr=2e-5, device="cuda"):
    model.to(device)
    model.train()

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    loss_fn = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")

        # Training loop
        total_loss = 0
        for batch in tqdm(train_dataloader, desc="Training"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            loss = loss_fn(logits, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_dataloader)
        print(f"Train Loss: {avg_loss:.4f}")

        # Optional: basic validation accuracy after each epoch
        val_acc = evaluate_model(model, val_dataloader, model_name="bert-base", device=device)["accuracy"]
        print(f"Validation Accuracy: {val_acc:.4f}")

In [ ]:
def train_on_increasing_sizes_simple(
    model_template_path,
    train_texts,
    train_labels,
    val_dataloader,
    tokenizer,
    fractions=[0.1, 0.25, 0.5, 0.75, 1.0],
    batch_size=16,
    epochs=3,
    device="cuda"
):
    results = {}

    # make sure data is in consistent order
    combined = list(zip(train_texts, train_labels))
    total = len(combined)

    for frac in fractions:
        num_examples = int(frac * total)
        print(f"\nTraining on {num_examples} examples ({int(frac * 100)}%)")

        # Take a simple slice of the data
        subset = combined[:num_examples]
        sub_texts, sub_labels = zip(*subset)

        # Tokenize and wrap into Dataset and DataLoader
        train_dataset = SentimentDataset(list(sub_texts), list(sub_labels), tokenizer)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        # Load a fresh model
        model = AutoModelForSequenceClassification.from_pretrained(model_template_path, num_labels=3)

        # Train and evaluate
        train_bert(model, train_loader, val_dataloader, epochs=epochs, device=device)
        metrics = evaluate_model(model, val_dataloader, model_name="bert-base", device=device)

        results[f"{int(frac*100)}%"] = metrics

    return results

In [ ]:
# Prepare val_loader (only once)
val_dataset = SentimentDataset(val_texts, val_labels, models["bert-base"]["tokenizer"])
val_loader = DataLoader(val_dataset, batch_size=16)

# Run training loop
results = train_on_increasing_sizes_simple(
    model_template_path="bert-base-uncased",
    train_texts=train_texts,
    train_labels=train_labels,
    val_dataloader=val_loader,
    tokenizer=models["bert-base"]["tokenizer"],
    fractions=[0.1, 0.25, 0.5, 0.75, 1.0],
    batch_size=16,
    epochs=3
)

In [ ]:
results

Now we can start to train the bert model on the dataset